In [1]:
import requests
import json
import time
from bs4 import BeautifulSoup
import pandas as pd
import datetime

In [2]:
username = ""
password = ""

contest_id = "ahc002"
problem_id = "ahc002_a"

contest_start = datetime.datetime(2022,12,2,19,0,0)
contest_end = datetime.datetime(2022,12,11,19,0,0)

In [3]:
personal = {
 'jabee': ['三尺秋水'],
 'keroru': ['Moréchond'],
 'rinrion': ['三尺秋水'],
 'kuruton456': ['Moréchond'],
 'kokatsu': ['ひよこ'],
 'kaede2020': ['Moréchond'],
 'michirakara': ['Moréchond'],
 'yas_yasyu': ['Moréchond'],
 'hiraku': ['焼きにゃまし隊'],
 'nephrologist': ['ひよこ'],
 'meowmeowcats': ['焼きにゃまし隊'],
 'gabriel55_': ['三尺秋水'],
 'G4NP0N': ['俺はタスマニア！お前は何マニア？'],
 'bio4eta': ['ひよこ'],
 'Gex777': ['三尺秋水'],
 'a9ua1i0n': ['a1nem'],
 'ainem': ['運営'],
 'ococonomy1': ['a1nem'],
 'amylase': ['俺はタスマニア！お前は何マニア？'],
 'uni_kakurenbo': ['a1nem'],
 'bk_cocoa': ['ひよこ'],
 'fujikawahiroaki': ['三尺秋水'],
 'akane831': ['焼きにゃまし隊'],
 'tardigrade': ['俺はタスマニア！お前は何マニア？'],
 'r3yohei': ['焼きにゃまし隊'],
 'takekinmkk': ['俺はタスマニア！お前は何マニア？']
}

In [4]:
for name in personal.keys():
  url = "https://atcoder.jp/users/" + name + "/history/json"
  json = requests.session().get(url).json()
  argo_rate = 0
  for data in json:
    ymd, hms_ = data["EndTime"].split("T")
    y,m,d = map(int,ymd.split("-"))
    h,min,sec = map(int,hms_.split("+")[0].split(":"))
    end_time = datetime.datetime(y,m,d,h,min,sec)
    if end_time > contest_start:
      break
    argo_rate = data["NewRating"]

  url = "https://atcoder.jp/users/" + name + "/history/json?contestType=heuristic"
  json = requests.session().get(url).json()
  heur_rate = 0
  heur_highest_perf = 0
  for data in json:
    ymd, hms_ = data["EndTime"].split("T")
    y,m,d = map(int,ymd.split("-"))
    h,min,sec = map(int,hms_.split("+")[0].split(":"))
    end_time = datetime.datetime(y,m,d,h,min,sec)
    if end_time > contest_start:
      break
    heur_rate = data["NewRating"]
    heur_highest_perf = max(heur_highest_perf, data["NewRating"])
  personal[name] += [argo_rate, heur_rate, heur_highest_perf]


In [5]:
df=pd.DataFrame.from_dict(personal, orient="index", columns=["チーム名","Aレート","Hレート","BestPerf"])
df.to_csv("personal_info.csv")

In [6]:
personal = pd.read_csv("personal_info.csv", index_col=0)
personal.head()

,チーム名,Aレート,Hレート,BestPerf
jabee,三尺秋水,1359,1035,1035
keroru,Moréchond,1607,1320,1471
rinrion,三尺秋水,471,1266,1266
kuruton456,Moréchond,1604,1494,1613
kokatsu,ひよこ,1258,921,921


In [ ]:
# User Agentの設定の設定
user_agent = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.4.1 Safari/605.1.15"
header = {
    'User-Agent': user_agent
}

login_url = "https://atcoder.jp/login?continue=https%3A%2F%2Fatcoder.jp%2F"
session = requests.session()
response = session.get(login_url, headers=header)

# BeautifulSoupオブジェクト作成(token取得の為)
bs = BeautifulSoup(response.text, 'html.parser')
# tokenの取得
csrf_token = bs.find(attrs={'name':'csrf_token'}).get('value')

login_data = {
   'username': username,
   'password': password,
   'csrf_token': csrf_token,
}

# cookieの取得
response_cookie = response.cookies

login = session.post(login_url, data=login_data, cookies=response_cookie)
time.sleep(2) # 少し時間を置いてみる

In [ ]:
#standings_url = "https://atcoder.jp/contests/" + contest_id + "/standings/json"
#standings_json = session.get(standings_url).json()

In [ ]:
def make_datetime(s: str) -> datetime:
  ymd, hms = s.split(" ")
  year, month, day = map(int,ymd.split("-"))
  hour, minute, second = hms.split(":")
  second = int(second.split("+")[0])

  res = datetime.datetime(year=year, month=month, day=day, hour=int(hour), minute=int(minute), second=int(second))

  return res

In [ ]:
records = {
    "name": [],
    "team": [],
    "lang": [],
    "score": [],
    "submit": [],
    "elapsed_time": [],
    #"argo_rate": [],
    #"heuristic_rate": [],
    #"performance": []
}

submission_url = "https://atcoder.jp/contests/" + contest_id +"/submissions?desc=true&f.LanguageName=&f.Status=AC&f.Task=" + problem_id + "&f.User=&orderBy=created&page="
page=1

not_in_record = set()

while True:
  submissions = session.get(submission_url + str(page))
  soup = BeautifulSoup(submissions.text, "html5lib")
  tr = soup.select("tbody > tr")
  if len(tr)==0:
    break
  for row in tr:
    submit = make_datetime(row.contents[1].text)
    if contest_start < submit < contest_end:
      name = row.contents[5].text.rstrip() if " " in row.contents[5].text else row.contents[5].text
      lang = row.contents[7].text
      score = int(row.find(name="td", attrs={"class": "submission-score"}).text)
      elapsed_time = row.contents[14].text
      if name in personal.index:
        tname = personal.at[name, "チーム名"]
        records["name"].append(name)
        records["team"].append(tname)
        records["lang"].append(lang)
        records["score"].append(score)
        records["submit"].append(submit)
        records["elapsed_time"].append(elapsed_time)
        #records["argo_rate"].append(a_rate)
        #records["heuristic_rate"].append(h_rate)
        #records["performance"].append(perf)
      else:
        not_in_record.add(name)
    elif submit < contest_start:
      break
  else:
    page += 1
    continue
  break

if not_in_record: print("バチャ期間中に提出があったが、参加者に登録されていない：", not_in_record)

バチャ期間中に提出があったが、参加者に登録されていない： {'motoshira', 'yu8589869056', 'machonium', 'nakashiii', 'mersnn621'}


In [ ]:
df = pd.DataFrame(data = records)
df

,name,team,lang,score,submit,elapsed_time
0,keroru,Moréchond,PyPy3 (7.3.0),6104095,2022-12-11 18:58:28,1988 ms
1,keroru,Moréchond,PyPy3 (7.3.0),6100764,2022-12-11 18:51:28,1989 ms
2,keroru,Moréchond,PyPy3 (7.3.0),6092961,2022-12-11 18:45:38,1989 ms
3,keroru,Moréchond,PyPy3 (7.3.0),6109592,2022-12-11 18:15:09,1990 ms
4,r3yohei,焼きにゃまし隊,Rust (1.42.0),6032343,2022-12-11 18:08:34,1999 ms
...,...,...,...,...,...,...
585,r3yohei,焼きにゃまし隊,Rust (1.42.0),774689,2022-12-02 20:05:11,196 ms
586,bio4eta,ひよこ,Python (3.8.2),4861,2022-12-02 19:52:25,26 ms
587,ococonomy1,a1nem,C++ (GCC 9.2.1),206679,2022-12-02 19:45:36,10 ms
588,ococonomy1,a1nem,C++ (GCC 9.2.1),7929,2022-12-02 19:20:25,6 ms


In [ ]:
df.to_csv("submissions.csv")